# Multi LInear Regression

## Data: FinalDataset.csv
## X: Urban spatial data of 500m around a subway station
## Y: From Fast Fourier Transformation, magnitude of each frequecy of every 10 interver from 0 to 100


In [2]:
import pandas as pd
import torch
from torch import nn
from torch import optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.base import BaseEstimator

date_types = ['wd_sc', 'wd_hc', 'we_sc', 'we_hc']
targets = ['cA', 'cD']
clusters = [['석촌', '태릉입구', '홍대입구', '강남', '선릉', '역삼', '교대', '공덕', '수유', '상봉', '중화', '상도']]

#y = np.log(df['max_ride'])

for date_type in date_types:
    df = pd.read_csv(f'../Data/240103_data/train_{date_type}.csv', index_col = 0)
    x = df.iloc[:, 2:-8]
    #y = df[target]

    x = x.fillna(0)
    for cluster_number in range(len(clusters)):
        test_stations = clusters[cluster_number]
        def str_to_arr(x):
            return np.array([float(i) for i in x[1:-1].strip().split()])

        test_id = df[df['name'].isin(test_stations)].index
        train_id = df[~df['name'].isin(test_stations)].index
        print(list(df[df['name'].isin(test_stations)]['name']))
        for i in range(2):
            target = targets[i]
            y = df[target]
            x_test = x.loc[test_id, :]
            x_train = x.loc[train_id, :]
            y_test = y.loc[test_id]
            y_train = y.loc[train_id]
            y_test_np = [str_to_arr(i) for i in y_test]
            y_train_np = [str_to_arr(i) for i in y_train]

            x_train_tensor = torch.tensor(x_train.values, dtype = torch.float)
            x_test_tensor = torch.tensor(x_test.values, dtype = torch.float)
            y_train_tensor = torch.tensor(y_train_np, dtype = torch.float)
            y_test_tensor = torch.tensor(y_test_np, dtype = torch.float)

            x_train, y_train = x_train_tensor, y_train_tensor
            x_test, y_test = x_test_tensor, y_test_tensor
            '''
            x_train = torch.tensor(x_train, dtype = torch.float)
            x_test = torch.tensor(x_test, dtype = torch.float)
            y_train = torch.tensor(y_train, dtype = torch.float)
            y_test = torch.tensor(y_test, dtype = torch.float)
            '''
            array_y_test = y_test.numpy()
            y_test = pd.DataFrame(array_y_test)
            y_test.to_csv(f'../Result/240116_LastCluster/Cluster{cluster_number+1}/test_{target}_{date_type}.csv')
            torch.cuda.empty_cache()

            class MultivariateLinearRegressionModel(nn.Module):
                def __init__(self, input_size = 17, hidden_size = 4, output_size = 11, dropout_prob = 0.3):
                    super().__init__()
                    self.linear1 = nn.Linear(input_size, hidden_size)
                    self.dropout1 = nn.Dropout(p = dropout_prob)
                    self.linear2 = nn.Linear(hidden_size, output_size)
                    self.dropout2 = nn.Dropout(p = dropout_prob)
                    self.relu = nn.ReLU()
                    self.sigmoid = nn.Sigmoid()

                def forward(self, x):
                    x = torch.sigmoid(self.linear1(x))
                    x = self.dropout1(x)
                    x = self.linear2(x)
                    return x
            model = MultivariateLinearRegressionModel()
            criterion1 = nn.MSELoss()
            #criterion2 = nn.L1Loss()
            optimizer = optim.SGD(model.parameters(), lr = 0.001)

            for epoch in range(150000):
                output = model(x_train)
                #cost = criterion1(output, y_train) + criterion2(output, y_train)
                cost = criterion1(output, y_train)

                optimizer.zero_grad()
                cost.backward()
                optimizer.step()

                if (epoch + 1) % 100 == 0:
                    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

            with torch.no_grad():
                predictions = model(x_test)

            array_y_pred = predictions.numpy()
            y_pred = pd.DataFrame(array_y_pred)
            y_pred.to_csv(f'../Result/240116_LastCluster/Cluster{cluster_number+1}/pred_{target}_{date_type}_hidden4.csv')

['태릉입구', '석촌', '강남', '상도', '역삼', '선릉', '홍대입구', '공덕', '중화', '상봉', '수유', '교대']
Epoch :  100, Cost : 0.289
Epoch :  200, Cost : 0.272
Epoch :  300, Cost : 0.257
Epoch :  400, Cost : 0.240
Epoch :  500, Cost : 0.236
Epoch :  600, Cost : 0.226
Epoch :  700, Cost : 0.218
Epoch :  800, Cost : 0.199
Epoch :  900, Cost : 0.189
Epoch : 1000, Cost : 0.187
Epoch : 1100, Cost : 0.176
Epoch : 1200, Cost : 0.176
Epoch : 1300, Cost : 0.169
Epoch : 1400, Cost : 0.158
Epoch : 1500, Cost : 0.160
Epoch : 1600, Cost : 0.150
Epoch : 1700, Cost : 0.149
Epoch : 1800, Cost : 0.144
Epoch : 1900, Cost : 0.140
Epoch : 2000, Cost : 0.128
Epoch : 2100, Cost : 0.124
Epoch : 2200, Cost : 0.129
Epoch : 2300, Cost : 0.119
Epoch : 2400, Cost : 0.122
Epoch : 2500, Cost : 0.118
Epoch : 2600, Cost : 0.114
Epoch : 2700, Cost : 0.119
Epoch : 2800, Cost : 0.109
Epoch : 2900, Cost : 0.110
Epoch : 3000, Cost : 0.108
Epoch : 3100, Cost : 0.109
Epoch : 3200, Cost : 0.109
Epoch : 3300, Cost : 0.101
Epoch : 3400, Cost : 0.101
Epoch

In [6]:
clusters = [['길동', '동작', '둔촌동', '몽촌토성', '반포', '석촌', '송정', '신길', '신설동', '양평', '태릉입구'],
            ['구로디지털단지', '신도림', '홍대입구', '신림','서울대입구', '강남'],
            ['가산디지털단지', '광화문', '삼성', '선릉', '성수', '신사', '압구정', '여의도', '역삼', '을지로입구', '종각'],
            ['경복궁', '공덕', '교대', '내방', '동대문역사공원', '뚝섬', '매봉', '문정', '발산', '신용산', '을지로3가', '청담', '충무로'],
            ['구의', '구파발', '길음', '낙성대', '노원', '목동', '미아사거리', '상봉', '상일동', '수유', '창동', '하계', '홍제'],
            ['공릉', '군자', '굽은다리', '남성', '답십리', '면목', '복정', '불광', '사가정', '상도', '상왕십리', '수락산', '신풍', '아현', '옥수', '온수', '우장산', '월곡', '장승배기', '중화', '화랑대']]

#y = np.log(df['max_ride'])


    
for cluster_number in range(6):
    df = pd.read_csv(f'../Data/240103_data/train_{date_type}.csv', index_col = 0)
    x = df.iloc[:, 2:-8]
    #y = df[target]

    x = x.fillna(0)
    test_stations = clusters[cluster_number]
    def str_to_arr(x):
        return np.array([float(i) for i in x[1:-1].strip().split()])

    test_id = df[df['name'].isin(test_stations)].index
    train_id = df[~df['name'].isin(test_stations)].index
    print(list(df[df['name'].isin(test_stations)]['name']))

['송정', '태릉입구', '석촌', '반포', '둔촌동', '몽촌토성', '동작', '신설동', '길동', '신길', '양평']
['서울대입구', '구로디지털단지', '신림', '강남', '홍대입구', '신도림']
['신사', '을지로입구', '여의도', '광화문', '압구정', '역삼', '선릉', '성수', '삼성', '종각', '가산디지털단지']
['문정', '매봉', '내방', '충무로', '발산', '뚝섬', '청담', '을지로3가', '공덕', '교대', '경복궁', '신용산']
['길음', '창동', '미아사거리', '하계', '구파발', '구의', '상봉', '상일동', '노원', '목동', '수유', '홍제', '낙성대']
['복정', '공릉', '답십리', '월곡', '사가정', '남성', '온수', '상왕십리', '장승배기', '상도', '옥수', '면목', '화랑대', '수락산', '군자', '중화', '굽은다리', '아현', '우장산', '불광', '신풍']


In [244]:
#model = nn.Linear(10, 10)
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self, input_size = 17, hidden_size = 4, output_size = 11, dropout_prob = 0.3):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(p = dropout_prob)
        self.linear2 = nn.Linear(hidden_size, output_size)
        self.dropout2 = nn.Dropout(p = dropout_prob)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = self.dropout1(x)
        x = self.linear2(x)
        return x
model = MultivariateLinearRegressionModel()
criterion1 = nn.MSELoss()
#criterion2 = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

for epoch in range(150000):
    output = model(x_train)
    #cost = criterion1(output, y_train) + criterion2(output, y_train)
    cost = criterion1(output, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :  100, Cost : 0.639
Epoch :  200, Cost : 0.599
Epoch :  300, Cost : 0.574
Epoch :  400, Cost : 0.523
Epoch :  500, Cost : 0.505
Epoch :  600, Cost : 0.473
Epoch :  700, Cost : 0.452
Epoch :  800, Cost : 0.421
Epoch :  900, Cost : 0.403
Epoch : 1000, Cost : 0.384
Epoch : 1100, Cost : 0.364
Epoch : 1200, Cost : 0.341
Epoch : 1300, Cost : 0.328
Epoch : 1400, Cost : 0.312
Epoch : 1500, Cost : 0.301
Epoch : 1600, Cost : 0.279
Epoch : 1700, Cost : 0.274
Epoch : 1800, Cost : 0.262
Epoch : 1900, Cost : 0.251
Epoch : 2000, Cost : 0.234
Epoch : 2100, Cost : 0.226
Epoch : 2200, Cost : 0.209
Epoch : 2300, Cost : 0.216
Epoch : 2400, Cost : 0.201
Epoch : 2500, Cost : 0.198
Epoch : 2600, Cost : 0.189
Epoch : 2700, Cost : 0.185
Epoch : 2800, Cost : 0.190
Epoch : 2900, Cost : 0.175
Epoch : 3000, Cost : 0.171
Epoch : 3100, Cost : 0.170
Epoch : 3200, Cost : 0.167
Epoch : 3300, Cost : 0.153
Epoch : 3400, Cost : 0.160
Epoch : 3500, Cost : 0.152
Epoch : 3600, Cost : 0.148
Epoch : 3700, Cost : 0.149
E

In [246]:
array_y_pred = predictions.numpy()
y_pred = pd.DataFrame(array_y_pred)
y_pred.to_csv(f'../Result/240103_Multi+Bayesian Quantity/pred_{target}_{date_type}_hidden4.csv')

'''array_y_test = y_test.numpy()
y_test = pd.DataFrame(array_y_test)
y_test.to_csv(f'../Result/231201_MultilinearRegression/test_{target}_{date_type}.csv')'''

"array_y_test = y_test.numpy()\ny_test = pd.DataFrame(array_y_test)\ny_test.to_csv(f'../Result/231201_MultilinearRegression/test_{target}_{date_type}.csv')"

In [ ]:
#model = nn.Linear(10, 10)
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self, hidden_size = 16, n_layers = 2, dropout_prob = 0.3):
        super().__init__()
        self.layers = nn.Sequential(*[nn.Linear(13, hidden_size), nn.Dropout(p = dropout_prob)] * n_layers)
        self.output = nn.Linear(hidden_size, 1)
        self.ReLU = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layers(x)
        x = self.ReLU(self.output(x))
        return self.sigmoid(x)
    
class SklearnCompatibleModel(BaseEstimator):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        
       
criterion1 = nn.MSELoss()
#criterion2 = nn.L1loss()
optimizer = optim.Adam

param_grid = {
    'hidden_size': [16, 32, 64, 128],
    'n_layers': [2, 3, 4],
    'learning_rate': [0.001, 0.0001, 0.00001],
}
model = MultivariateLinearRegressionModel()
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs = -1)

grid_search.fit(x_train, y_train)

# Print the best hyperparameter configuration and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

# Evaluate the model with the best hyperparameters on test data
best_model = grid_search.best_estimator_
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

In [ ]:
#model = nn.Linear(10, 10)
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self, input_size = 13, output_size = 12, dropout_prob = 0.3):
        super().__init__()
        self.linear1 = nn.Linear(input_size, 8)
        self.dropout1 = nn.Dropout(p = dropout_prob)
        self.linear2 = nn.Linear(8, 4)
        self.dropout2 = nn.Dropout(p = dropout_prob)
        self.linear3 = nn.Linear(4, 8)
        self.dropout3 = nn.Dropout(p = dropout_prob)
        self.linear4 = nn.Linear(8, 10)
        self.dropout4 = nn.Dropout(p = dropout_prob)
        self.linear5 = nn.Linear(10, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = self.dropout1(x)
        x = torch.sigmoid(self.linear2(x))
        x = self.dropout2(x)
        x = torch.sigmoid(self.linear3(x))
        x = self.dropout3(x)
        x = torch.sigmoid(self.linear4(x))
        x = self.dropout4(x)
        x = self.linear5(x)
        return x
model = MultivariateLinearRegressionModel()
criterion1 = nn.MSELoss()
criterion2 = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

for epoch in range(10000):
    output = model(x_train)
    #cost = criterion1(output, y_train) + criterion2(output, y_train)
    peak_gradient = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
    output2 = torch.from_numpy(np.dot(output.detach().numpy(), peak_gradient))
    y_train2 = torch.from_numpy(np.dot(y_train.detach().numpy(), peak_gradient))
    cost = criterion1(output, y_train) + criterion2(output2, y_train2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

In [ ]:
#model = nn.Linear(10, 10)
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self, input_size = 13, output_size = 1, dropout_prob = 0.3):
        super().__init__()
        self.linear1 = nn.Linear(input_size, 8)
        self.dropout1 = nn.Dropout(p = dropout_prob)
        self.linear2 = nn.Linear(8, 4)
        self.dropout2 = nn.Dropout(p = dropout_prob)
        self.linear3 = nn.Linear(4, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.linear(x)
        return x
    
model = MultivariateLinearRegressionModel()
criterion1 = nn.MSELoss()
#criterion2 = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

for epoch in range(20000):
    output = model(x_train)
    #cost = criterion1(output, y_train) + criterion2(output, y_train)
    cost = criterion1(output, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

In [ ]:
predictions

In [ ]:
print(criterion1(predictions, y_test) + criterion2(predictions, y_test))
predictions

In [ ]:
array = predictions.numpy()
predictions = pd.DataFrame(array)
predictions.to_csv('cA_result_MSE.csv')

In [ ]:
y_test